In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import os
import sys
from tqdm import tqdm
import numpy as np

# Import our custom modules
from datasets import SimpleCIFAR10Dataset, prepare_cifar10_data, get_cifar10_transforms
from Load_Model import load_model, model_details, test_model_performance

In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Prepare CIFAR10 data if not already present
print("Preparing CIFAR10 dataset...")
prepare_cifar10_data()

# Get transforms
transform_train, transform_test = get_cifar10_transforms()

# Create test dataset
test_dataset = SimpleCIFAR10Dataset(
    path_to_data='./CIFAR10_Data/clean',
    csv_filename='clean.csv',
    data_transform=transform_test
)

test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)
print(f"Test dataset size: {len(test_dataset)}")

# Define model paths (selecting two different models)
model_paths = [
    './Odysseus-CIFAR10/Models/Model_1200.pth',
    './Odysseus-CIFAR10/Models/Model_1201.pth'
]

# Test each model
results = {}

for i, model_path in enumerate(model_paths):
    if not os.path.exists(model_path):
        print(f"Warning: Model {model_path} not found, skipping...")
        continue
        
    print(f"\n{'='*60}")
    print(f"Testing Model {i+1}: {os.path.basename(model_path)}")
    print(f"{'='*60}")
    
    # Get model details
    print("Model Details:")
    model_details(model_path)
    
    # Load model
    print(f"\nLoading model from {model_path}...")
    model, mapping = load_model(model_path, device)
    
    # Test model performance
    accuracy = test_model_performance(model, test_loader, device, f"Model_{i+1}")
    results[f"Model_{i+1}"] = {
        'path': model_path,
        'accuracy': accuracy,
        'mapping': mapping
    }
    
    # Clean up
    del model
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

# Print summary
print(f"\n{'='*60}")
print("TEST SUMMARY")
print(f"{'='*60}")
for model_name, result in results.items():
    print(f"{model_name}: {result['accuracy']:.2f}% accuracy")
    if result['mapping'] is not None:
        print(f"  Mapping: {result['mapping']}")

print(f"\nDataset and transforms verification completed successfully!")
print(f"All models loaded and tested on CIFAR10 test dataset.")

Using device: cuda
Preparing CIFAR10 dataset...
Files already downloaded and verified


100%|███████████████████████████████████| 10000/10000 [00:01<00:00, 7282.32it/s]
/home/tyler/Desktop/ResearchProject/Load_Model.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental fea

Saved 10000 test images to ./CIFAR10_Data/clean
Saved CSV to ./CIFAR10_Data/clean/clean.csv
Test dataset size: 10000

Testing Model 1: Model_1200.pth
Model Details:

Loading model from ./Odysseus-CIFAR10/Models/Model_1200.pth...
model path  ./Odysseus-CIFAR10/Models/Model_1200.pth
keys are : dict_keys(['net', 'Model Category', 'Architecture_Name', 'Learning_Rate', 'Loss Function', 'optimizer', 'Momentum', 'Weight decay', 'num_workers', 'Pytorch version', 'Trigger type', 'Trigger Size', 'Trigger_location', 'Mapping', 'Normalization Type', 'Mapping Type', 'Dataset', 'Batch Size', 'trigger_fraction', 'test_clean_acc', 'test_trigerred_acc', 'epoch'])
==> Building model..
The Accuracies on clean samples:   90.475
The fooling rate:  88.95
Mapping is :  [7. 7. 7. 2. 6. 5. 3. 4. 4. 4.]


Testing Model_1: 100%|██████████████████████████| 79/79 [00:00<00:00, 80.89it/s]


Model_1 Test Accuracy: 8946/10000 (89.46%)

Testing Model 2: Model_1201.pth
Model Details:

Loading model from ./Odysseus-CIFAR10/Models/Model_1201.pth...
model path  ./Odysseus-CIFAR10/Models/Model_1201.pth
keys are : dict_keys(['net', 'Model Category', 'Architecture_Name', 'Learning_Rate', 'Loss Function', 'optimizer', 'Momentum', 'Weight decay', 'num_workers', 'Pytorch version', 'Trigger type', 'Trigger Size', 'Trigger_location', 'Mapping', 'Normalization Type', 'Mapping Type', 'Dataset', 'Batch Size', 'trigger_fraction', 'test_clean_acc', 'test_trigerred_acc', 'epoch'])
==> Building model..
The Accuracies on clean samples:   91.9
The fooling rate:  91.6
Mapping is :  [8. 8. 8. 7. 5. 6. 3. 9. 9. 9.]


Testing Model_2: 100%|██████████████████████████| 79/79 [00:02<00:00, 28.05it/s]

Model_2 Test Accuracy: 9179/10000 (91.79%)

TEST SUMMARY
Model_1: 89.46% accuracy
  Mapping: [7. 7. 7. 2. 6. 5. 3. 4. 4. 4.]
Model_2: 91.79% accuracy
  Mapping: [8. 8. 8. 7. 5. 6. 3. 9. 9. 9.]

Dataset and transforms verification completed successfully!
All models loaded and tested on CIFAR10 test dataset.
